In [2]:
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image
from flask import request, jsonify, Flask
from keras.models import Sequential, load_model
from keras import backend as k
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Create Flask app
app = Flask(__name__)

In [4]:
# Function to load the models
def load_model_fn(path):
    model = load_model(path)
    print('Model loaded!')
    return model

load = load_model_fn('./model.keras') 

Model loaded!


In [5]:
# Function to preprocess the image
def preprocess_image(image_path, target_size):
    try:
        # Load the image
        image = Image.open(image_path)

        # Convert the image to RGB
        rgb_image = image.convert("RGB")

        # Resize the RGB image
        resized_image = rgb_image.resize(target_size)

        # Convert the RGB image to a NumPy array
        image_array = np.array(resized_image)

        # Normalize the image by dividing pixel values by 255
        mean_value = 0.5
        std_value = 0.5
        image_array = (image_array - mean_value) / std_value

        # Add batch dimension
        image_array = np.expand_dims(image_array, axis=0)

        return image_array
    except Exception as e:
        print(f"Error preprocessing image: {str(e)}")
        return None

In [7]:
classes = [
                "bamboo",
                "candles",
                "carpets",
                "crocheting",
                "jewelry",
                "leather",
                "pottery",
                "sewing",
                "soap",
                "wooden",
            ]

In [8]:
@app.route('/predictApi', methods=['POST'])
def api():
    try:
        # Check if the file exists in the request
        if 'upload' not in request.files:
            return jsonify({"error": "Please upload an image."})

        # Read the image file from the request
        image = request.files.get('upload')

        # Preprocess the image
        image_arr = preprocess_image(image, target_size=(224, 224))

        # Handle error if image preprocessing fails
        if image_arr is None:
            return jsonify({"error": "Error processing image."})

        # Make predictions using the loaded model
        prediction = load.predict(image_arr)

        # Get the predicted class label index and confidence
        index_largest_value = np.argmax(prediction[0])
        confidence = prediction[0][index_largest_value]

        # Check if the confidence is below the threshold
        threshold = 0.7  # Set your desired confidence threshold here
        if confidence < threshold:
            return jsonify({"error": "Confidence below threshold."})
            #return jsonify({"error": "Photo not found."})

        result = classes[index_largest_value]

        # Check if the predicted class label is in the classes list
        if result not in classes:
            return jsonify({"error": "Photo not found."})

        # Return the prediction and confidence as JSON response
        return jsonify({"prediction": result, "confidence": float(confidence)})
    except Exception as e:
        # Return error message as JSON response
        return jsonify({"error": str(e)})

In [ ]:
if __name__ == "__main__":
    # Run the Flask app
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/May/2024 18:47:09] "POST / HTTP/1.1" 404 -
